In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pickle
from steenroder import barcodes

In [ ]:
with open('notebooks/rp4.pkl', 'rb') as f:
    filtration = pickle.load(f)

In [ ]:
filtration = (
    (0,), 
    (1,), (0,1), 
    (2,), (0,2), (1,2), (0,1,2),
    (3,), (0,3), (1,3), (0,1,3), (2,3),
    (4,), (0,4), (1,4), (2,4), (1,2,4), (3,4), (0,3,4), (2,3,4),
    (5,), (0,5), (1,5), (2,5), (0,2,5), (3,5), (1,3,5), (2,3,5), (4,5), (0,4,5), (1,4,5)
    )

# New approach

In [ ]:
# %%timeit
barcodes(1, filtration)[1]

# Using the previous approach

In [ ]:
barcodes(1, filtration)[1]